### 1、tensorflow 1.x 保存 MNIST 分类模型

本文档在 tensorflow 1.x 的环境下训练一个mnist数据集分类模型，并保存训练之后的模型，作为后续部署的模型。
由于本项目主要目的是说明模型的部署，所以模型和训练尽可能简单，理论上，任意模型按照本文档的方式保存，均可以进行后续的部署。

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 获取 mnist数据
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
mnist_train,mnist_validation,mnist_test=mnist.train,mnist.validation,mnist.test

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
from tensorflow import graph_util # 保存模型用到的方法

# 模型参数
input_dim=784
hidden_dim=200
output_dim=10
learning_rate=0.5
train_steps=10000
batch_size=100

# 模型的输入，在调用部署后的模型时需要用到，所以这里最好自己命名
x = tf.placeholder(tf.float32, [None, input_dim], name='x-input')
y_ = tf.placeholder(tf.float32, [None, output_dim], name='y-input')
print(x)

# 构建一个三层的全连接层网络，输入层、隐层和输出层分别为784,200,10
weights_1 = tf.Variable(tf.random_normal(shape=[input_dim,hidden_dim],stddev=0.1),name='w1')
bias_1 = tf.Variable(tf.constant(0.0, shape=[hidden_dim]),name="b1")
hidden=tf.matmul(x,weights_1)+bias_1
hidden=tf.nn.relu(hidden)

weights_2 = tf.Variable(tf.random_normal(shape=[hidden_dim,output_dim],stddev=0.1),name='w2')
bias_2 = tf.Variable(tf.constant(0.0, shape=[output_dim]),name="b2")
y=tf.matmul(hidden,weights_2)+bias_2

# 模型的预测输出，注意，因为我们部署之后希望拿到这个值，所以这里最好自己命名，后面部署之后的调用会用到
soft_y=tf.nn.softmax(y,name="predict") 


# 模型损失
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
loss = tf.reduce_mean(cross_entropy)

# 预测精度
correct_predict=tf.equal(tf.math.argmax(y,1),tf.math.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_predict,tf.float32))

#模型训练
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    validation_feed={x:mnist_validation.images,y_:mnist_validation.labels}
    test_feed={x:mnist_test.images,y_:mnist_test.labels}
    for i in range(train_steps):
        train_samples,train_labels=mnist_train.next_batch(batch_size)
        _,train_loss=sess.run([train_step,loss],feed_dict={x:train_samples,y_:train_labels})
        if i%1000==0:
            val_acc=sess.run(accuracy,feed_dict=validation_feed)
            print("after %d training steps, loss of train set is %f,validation accuracy is %f"%(i,train_loss,val_acc))
    test_acc=sess.run(accuracy,feed_dict=test_feed)
    print("test accuracy is %f"%test_acc)
    
    # 保存模型
    # 将权重和偏置等变量转换为常量，然后将计算图保存，可以大大减小保存后的模型
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def,output_node_names=["predict"]) 
    model_save_path="model_save/mnist_classification.pb"
    with tf.gfile.FastGFile(model_save_path, mode='wb') as f:
        f.write(constant_graph.SerializeToString())

Tensor("x-input:0", shape=(?, 784), dtype=float32)
Instructions for updating:
Colocations handled automatically by placer.
after 0 training steps, loss of train set is 2.865034,validation accuracy is 0.258800
after 1000 training steps, loss of train set is 0.112137,validation accuracy is 0.971600
after 2000 training steps, loss of train set is 0.101401,validation accuracy is 0.977800
after 3000 training steps, loss of train set is 0.029216,validation accuracy is 0.979200
after 4000 training steps, loss of train set is 0.015400,validation accuracy is 0.979200
after 5000 training steps, loss of train set is 0.029765,validation accuracy is 0.983800
after 6000 training steps, loss of train set is 0.011433,validation accuracy is 0.982400
after 7000 training steps, loss of train set is 0.014701,validation accuracy is 0.981000
after 8000 training steps, loss of train set is 0.005434,validation accuracy is 0.981200
after 9000 training steps, loss of train set is 0.010085,validation accuracy is

In [4]:
# 在python 中调用保存好的模型进行预测
from tensorflow.python.platform import gfile

model_save_path="model_save/mnist_classification.pb"

with tf.Session() as sess:
    # 读取保存的模型
    with gfile.FastGFile(model_save_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def)
    tf.global_variables_initializer().run()
    g = tf.get_default_graph()
    y_pred=g.get_tensor_by_name("predict:0")
    x = g.get_tensor_by_name("x-input:0")
    y_= sess.run(y_pred, feed_dict={x: mnist_test.images})
    print(y_)

[[0.07017156 0.06350049 0.01443778 ... 0.34903672 0.09104601 0.07863592]
 [0.23315549 0.07266916 0.02236856 ... 0.09671797 0.15844405 0.12381011]
 [0.27223897 0.08200911 0.06456126 ... 0.09693536 0.11303534 0.1633693 ]
 ...
 [0.13931967 0.03895352 0.05266058 ... 0.41981813 0.06385793 0.06327698]
 [0.26489916 0.06770544 0.05480701 ... 0.13090087 0.18384087 0.08028858]
 [0.31137165 0.28877038 0.01396287 ... 0.18734947 0.02521153 0.02698081]]
